# Pandas Overview
Prepared by: Nick Freeman, PhD

This notebook provides a brief introduction to the `pandas` library that is available for Python. From https://en.wikipedia.org/wiki/Pandas_(software):

> pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series. It is free software released under the three-clause BSD license. The name is derived from the term "panel data", an econometrics term for data sets that include observations over multiple time periods for the same individuals.
>
>[Features of the library include:]
>
> - DataFrame object for data manipulation with integrated indexing.
> - Tools for reading and writing data between in-memory data structures and different file formats.
> - Data alignment and integrated handling of missing data.
> - Reshaping and pivoting of data sets.
> - Label-based slicing, fancy indexing, and subsetting of large data sets.
> - Data structure column insertion and deletion.
> - Group by engine allowing split-apply-combine operations on data sets.
> - Data set merging and joining.
> - Hierarchical axis indexing to work with high-dimensional data in a lower-dimensional data structure.
> - Time series-functionality: Date range generation and frequency conversion, moving window statistics, moving window linear > - regressions, date shifting and lagging.
> - Provides data filtration.
>
>The library is highly optimized for performance, with critical code paths written in Cython or C.

We will demonstrate how to work with the `pandas` library using a dataset that we will also use later when considered the concept of spend analysis. The data, which was downloaded from https://usaspending.org, includes records of government expenditures by the state of Alabama durng fiscal year 2019 and is provided in a *comma-separated value* format. By default, most machines that have Microsoft Office installed will open such files with Excel. We will use the `pandas` library to work with the data. We will also use the `pathlib` library, which is included as part of Python's standard library, to inspect files on our machine. the following code block imports the two libraries. Notices that we use the `as` import method with `pandas` so that we can give the library an alias (of `pd`).

In [ ]:
import pathlib

import pandas as pd

The following code block sets some options for pandas. In particular, we specify 1) that we want to display 40 columns of any imported data and 2) that we want to display floating point data with up to four decimal points.

In [ ]:
# Specify maximum columns = 40
pd.set_option('display.max_columns', 40)

# Specify floating-point precision
pd.set_option('display.float_format', '{:.4f}'.format)

Our data file is stored in a folder named `data` that resides in the same directory as this notebook. In particular, the name of the data file is `AL_FY2019.csv`. The following code block shows how we can use the `pathlib` library to create a `Path` object that points to the data file. We then use this path object to ensure that the file exists. If it does, we use the `read_csv` function that is included with the `pandas` library to read the data file.

**Note: In lieu of using the path object, you can simply provide the file path as a string to the `read_csv` function. For example, we can read our data file on a Windows machine using the syntax tha follows. I like to use the `pathlib.Path` approach because it offers some other functionality (like the `exists` method) and will figure out the correct path representation for a file, which can be different based on the operating system you are using, i.e., Windows vs. Mac vs. Linux.**

```python
# Alternate reading syntax
pd.read_csv('data/AL_FY2019.csv')
```

In [ ]:
data_filepath = pathlib.Path('data', 'AL_FY2019.csv')

if data_filepath.exists():
    print('File exists, reading with pandas.')
    data = pd.read_csv(data_filepath)
else:
    print('File does not exist!')

The following code block shows how we can check the `type` of a variable or object. In python, different types will have different `methods` and `attributes` available. Thus, it is important to understand the type of variable or object you are dealing with so that you know what options are available.

In [ ]:
type(data)

The previous code block shows that the `data` variable is actually a `pandas` `DataFrame`. DataFrames have a `head` method that prints the first five rows of data by default.

In [ ]:
data.head()

We can get more or less rows to print by supplying the optional argument `n`.

In [ ]:
data.head(n = 10) # or just data.head(10)

Similar to the layout of data in Excel, a `pandas` `DataFrame` has row and column labels. The row labels of a `pandas` `DataFrame` are given in the `index` and the column labels in the `columns`. We can access these labels using the `index` and `columns` attributes.

In [ ]:
data.index

In [ ]:
data.columns

Notice that the type of the object returned by both of these attributes is some type of an Index.

In [ ]:
type(data.index)

In [ ]:
type(data.columns)

Pandas index types have a `tolist` method that allows us to convert the index to a Python list.

In [ ]:
data.columns.tolist()

There is also a `describe` method available for `DataFrame` objects. By default, it summarizes numerical columns only.

In [ ]:
data.describe()

When we look at the concept of spend analysis, we will be using the `total_dollars_obligated` values as a proxy for expenditures. Notice in the previous summary that this data includes negative values, which may mean that some entity owes the government money. Suppose we want to exclude any non-positive values. To do this we first retrieve the column.

In [ ]:
data['total_dollars_obligated']

Just as an aside, note that the column is a `pandas` `series`.

In [ ]:
type(data['total_dollars_obligated'])

Pandas is built *on top* of another popular library named `numpy`. The primary objective of `numpy` is to allow for fast numerical computations on matrices or vectors (it is actually a Python wrapper for many fast implementations in C). The presence of `numpy` can be observed if we try to determine which rows include values less than or equal to zero.

In [ ]:
data['total_dollars_obligated'] <= 0

Note that the comparison provides a pandas series with `True` or `False` for each of the index (row) values.

In [ ]:
type(data['total_dollars_obligated'] <= 0)

Since the result is a series, we can save it as variable.

In [ ]:
mask = data['total_dollars_obligated'] <= 0
mask

With the series saved, we can use it to quickly subset and overwrite the data. Note that the `~` symbol represents a logical *not* statement. Thus, since our mask grabs index values where the `total_dollars_obligated` is less than or equal to zero, the statement `~mask` returns a series where index values corresponding to `total_dollars_obligated` values that are greater than zero are set to `True`.

In [ ]:
data = data[~mask]
data.head()

We can use the `reset_index` method to reset the index.

In [ ]:
data = data.reset_index(drop = True)
data.head()

The `pandas` library allows to quickly perform computations or get information for columns of a `DataFrame` or the entire `DataFrame` itself. The following code block prints the methods and attributes that are available for the `total_dollars_obligated` column. 

In [ ]:
print([item for item in dir(data['total_dollars_obligated']) if not item.startswith('_')])

A few example uses of the listed methods and attribues applied to the `total_dollars_obligated` column are printed in the following code block. Notice how I am able to *chain* methods together.

In [ ]:
print(f"The 5 largest values are:\n {data['total_dollars_obligated'].nlargest(5).values}\n")

print(f"The 5 smalles values are:\n {data['total_dollars_obligated'].nsmallest(5).values}\n")

print(f"The sum of the values is:\n {data['total_dollars_obligated'].sum()}\n")

print(f"The median of the values is:\n {data['total_dollars_obligated'].median()}\n")

The final thing that we will look at is the aggregation abilities available in `pandas`. In particular, `pandas` `DataFrame` objects have a groupby *method* that allows you to perform a wide array of aggregations in a simple manner. Although there is more than one way to use this method, the following code block shows how to determine the minimum, median, and maximum dollars obligated by awarding agency. I like the format because I find it very clear from a reader's perspective. The format uses a Python list to define the columns we are grouping on and a dictionary to specify the aggregations.

In [ ]:
groupby_columns = ['awarding_agency_name']

agg_dict = {
    'total_dollars_obligated': ['min', 'median', 'max'],
}

data.groupby(groupby_columns).agg(agg_dict)

The following code block prints different aggregations that we can use in the `agg_dict`. 

In [ ]:
print([item for item in dir(data.groupby(groupby_columns)['total_dollars_obligated']) if not item.startswith(('_', 'agg'))])